In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- OR summary ---
    if "OR" in d.columns:
        or_mean = d["OR"].mean()
        or_std = d["OR"].std(ddof=1)
        df_out.loc[feature, "OR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minOR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxOR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["OR"].mean()
    or_std  = d["OR"].std(ddof=1)
    df_out.loc[idx, "OR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minOR"] = d["2.5%"].min()
    df_out.loc[idx, "maxOR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
import time

t0 = time.perf_counter()






In [4]:
data = pd.read_csv('data/BBAG-cross.csv')

In [5]:
data = data[data.country != 'Slovakia'].reset_index(drop =  True)

In [6]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


# Odds ratios

## Without co-vars

### Without iterations

In [7]:
results_merge_df_all = data.copy()

results_merge_df_all = results_merge_df_all.loc[:, ~results_merge_df_all.columns.duplicated()]

df_directions_odd = pd.DataFrame()

for i in vars_:
    
    y_ols = results_merge_df_all['GAP_bin']

    
    X_ols = results_merge_df_all[[i]]
    X_train, X_test, y_train, y_test = train_test_split(X_ols, y_ols, test_size=0.2, random_state=42)
    

    scaler = MinMaxScaler((0.05, 0.95))
    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    X_train_scaled['intercept'] = 1
    X_test_scaled['intercept'] = 1
    
    model = sm.Logit(y_train, X_train_scaled).fit(disp = 0)
    
    params = model.params
    conf = np.exp(model.conf_int())
    conf['OR'] = np.exp(params)
    conf['z'] =model.tvalues
    conf['P>|z|'] =model.pvalues
    conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']
    
    df = conf.loc[i:i]
    df['Feature'] = i
    df_directions_odd = pd.concat([df_directions_odd, df])

df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd.to_excel('Results/cross/cross_OR_-removing-slovakia.xlsx')
df_directions_odd

,2.5%,97.5%,OR,z,P>|z|,Feature
0,1.994677,2.258758,2.122614,23.729241,1.800084e-124,Mono
1,0.701331,0.819006,0.757888,-7.005826,2.455334e-12,One
2,0.488677,0.567118,0.526438,-16.894970,4.899618e-64,Two
3,0.622843,0.722756,0.670942,-10.514530,7.404916e-26,Three
4,0.442640,0.501288,0.471052,-23.716052,2.462736e-124,Total


### 1000-iteration

In [8]:


count = 0

results_dict = {}
for i in vars_:
    results_dict[i + "_y"] = pd.Series(dtype=float)
    results_dict[i + "_ypred"] = pd.Series(dtype=float)

results_merge_df_all = data.copy()
results_merge_df_all.dropna(subset=vars_ + ["GAP_bin"], inplace=True)
results_merge_df_all.reset_index(inplace=True, drop=True)

df_directions_odd = pd.DataFrame()

n_boosts = 10  #For performance reasons, the number of iterations was set to 10; however, 1,000 iterations were used for the results reported in the manuscript.”
test_size = 500 / results_merge_df_all.shape[0]

for boosts in range(n_boosts):
    for i in vars_:

        y_ols = results_merge_df_all["GAP_bin"]
        X_ols = results_merge_df_all[[i]]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols,
            test_size=test_size,
            stratify=results_merge_df_all["GAP_bin"],
            random_state=boosts
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled["intercept"] = 1
        X_test_scaled["intercept"] = 1


        model = sm.Logit(y_train, X_train_scaled).fit(disp=0)


        y_test_pred = model.predict(X_test_scaled)

        results_dict[i + "_y"] = pd.concat([results_dict[i + "_y"], y_test], axis=0)
        results_dict[i + "_ypred"] = pd.concat([results_dict[i + "_ypred"], y_test_pred], axis=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf["OR"] = np.exp(params)
        conf["z"] = model.tvalues
        conf["P>|z|"] = model.pvalues
        conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

        df = conf.loc[i:i].copy()
        df["Feature"] = i

        hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
        hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
        hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

        df["hl_statistic_10"] = hl10_stat
        df["hl_pvalue_10"] = hl10_p

        df["hl_statistic_50"] = hl50_stat
        df["hl_pvalue_50"] = hl50_p

        df["hl_statistic_100"] = hl100_stat
        df["hl_pvalue_100"] = hl100_p

        df_directions_odd = pd.concat([df_directions_odd, df], axis=0)

    df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd;


In [9]:
df_directions_odd_f = pd.DataFrame()

for f in vars_:
    df_directions_odd_f = summarize_feature(df_directions_odd, f, df_directions_odd_f, stat_cols=(10, 50, 100))

df_directions_odd_f.to_excel('Results/cross/cross_OR_-removing-slovakia-iter.xlsx')
df_directions_odd_f

,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR
Mono,0.297950,0.400736,0.993578,2.133458,2.130743,2.136174,2.012721,2.259653
One,0.148495,0.311214,0.989381,0.758080,0.756058,0.760102,0.704368,0.817247
Two,0.158351,0.392823,0.996717,0.534625,0.533441,0.535808,0.498634,0.573086
Three,0.097559,0.460489,0.994873,0.659431,0.658065,0.660797,0.614173,0.706897
Total,0.304208,0.380079,0.994973,0.468607,0.468009,0.469205,0.442413,0.496737


## Odd ratios with macrosocials

### Without Iteration

In [10]:
vars_social_physical = ['gender_equal_l', 'Polution_conc_inv', 'Eq']

vars_sociopolitical = ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est']

In [11]:
results_merge_df_all = data.copy()

covar_list = (
    ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
     'number_leng_inst', 'number_stable_inst', 'distance', 'Migration']
    + vars_social_physical
    + vars_sociopolitical
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:

        y_ols = results_merge_df_all['GAP_bin']
        X_ols = results_merge_df_all[[i, covar]]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.Logit(y_train, X_train_scaled).fit(disp=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['OR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          # opcional pero útil
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [12]:
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    c_dfOR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-removing-slovakia_covar-' + covar + '.xlsx')

    print(covar)
    display(c_dfOR)

total_exposomes


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
0,2.216068,2.540576,2.372781,24.785296,1.291603e-135,Mono,total_exposomes
1,0.669424,0.785507,0.725147,-7.878026,3.325947e-15,One,total_exposomes
2,0.450516,0.530242,0.488756,-17.222580,1.797929e-66,Two,total_exposomes
3,0.600513,0.706678,0.651436,-10.319898,5.728420e-25,Three,total_exposomes
4,0.393448,0.451116,0.421296,-24.773725,1.721290e-135,Total,total_exposomes


sociopolitical


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
5,2.100860,2.382509,2.237257,25.090150,6.370388e-139,Mono,sociopolitical
6,0.709620,0.828785,0.766891,-6.702153,2.053703e-11,One,sociopolitical
7,0.473146,0.549778,0.510024,-17.582171,3.374042e-69,Two,sociopolitical
8,0.590365,0.686243,0.636501,-11.767511,5.739306e-32,Three,sociopolitical
9,0.419510,0.475800,0.446769,-25.084146,7.407782e-139,Total,sociopolitical


social_physical


...

,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
60,1.936755,2.196936,2.062747,22.516326,2.872192e-112,Mono,inclu_suff_est
61,0.714863,0.835013,0.772606,-6.509463,7.542002e-11,One,inclu_suff_est
62,0.511823,0.596444,0.552515,-15.199300,3.574193e-52,Two,inclu_suff_est
63,0.620704,0.720432,0.668712,-10.586731,3.433736e-26,Three,inclu_suff_est
64,0.455101,0.516280,0.484726,-22.505833,3.639153e-112,Total,inclu_suff_est


cred_elect_est


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
65,2.127855,2.419246,2.268878,25.023416,3.400306e-138,Mono,cred_elect_est
66,0.705426,0.825233,0.762982,-6.760186,1.378151e-11,One,cred_elect_est
67,0.444409,0.519669,0.480568,-18.360680,2.711895e-75,Two,cred_elect_est
68,0.600285,0.698969,0.647750,-11.183995,4.884069e-29,Three,cred_elect_est
69,0.413205,0.469839,0.440613,-25.012307,4.491747e-138,Total,cred_elect_est


local_dem_est


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
70,1.981132,2.245055,2.108969,23.388863,5.548201e-121,Mono,local_dem_est
71,0.711187,0.831110,0.768814,-6.613577,3.751418e-11,One,local_dem_est
72,0.492639,0.571846,0.530767,-16.654084,2.826391e-62,Two,local_dem_est
73,0.624122,0.724157,0.672281,-10.470230,1.183547e-25,Three,local_dem_est
74,0.445339,0.504717,0.474099,-23.374600,7.749025e-121,Total,local_dem_est


### 1000-Iteration

In [13]:

results_merge_df_all = data.copy()

covar_list = (
    ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
     'number_leng_inst', 'number_stable_inst', 'distance', 'Migration']
    + vars_social_physical
    + vars_sociopolitical
)



n_boosts = 10  #For performance reasons, the number of iterations was set to 10; however, 1,000 iterations were used for the results reported in the manuscript.”
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:
    print(covar)
    for boosts in range(n_boosts):

        for i in vars_:

            y_ols = results_merge_df_all["GAP_bin"]
            X_ols = results_merge_df_all[[i, covar]]

            # Split igual al primero: test fijo ~500 y estratificado
            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.Logit(y_train, X_train_scaled).fit(disp=0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["OR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


total_exposomes
sociopolitical
social_physical
GDP
number_leng_inst
number_stable_inst
distance
Migration
gender_equal_l
Polution_conc_inv
Eq
free_parties_l
inclu_suff_est
cred_elect_est
local_dem_est


In [14]:

features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100),
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [15]:
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    c_dfOR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-removing-slovakia_covar-' + covar + '-iter.xlsx')

    print(covar)
    display(c_dfOR)

total_exposomes


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
0,total_exposomes,Mono,0.420321,0.434801,0.992659,2.398779,2.393994,2.403564,2.249463,2.558376,10.0
1,total_exposomes,One,0.242979,0.393734,0.987029,0.727369,0.725426,0.729313,0.674243,0.785917,10.0
2,total_exposomes,Two,0.103535,0.279654,0.984770,0.495165,0.493644,0.496687,0.457569,0.535330,10.0
3,total_exposomes,Three,0.281401,0.219112,0.972813,0.635634,0.634143,0.637125,0.588043,0.685003,10.0
4,total_exposomes,Total,0.407806,0.455002,0.994347,0.416674,0.415842,0.417506,0.390660,0.444355,10.0


sociopolitical


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
5,sociopolitical,Mono,0.230993,0.362202,0.988885,2.252875,2.249745,2.256004,2.123875,2.387521,10.0
6,sociopolitical,One,0.082658,0.235533,0.966780,0.767077,0.765038,0.769117,0.712637,0.827040,10.0
7,sociopolitical,Two,0.169137,0.386922,0.989758,0.517141,0.515882,0.518400,0.481803,0.554939,10.0
8,sociopolitical,Three,0.284894,0.305681,0.976776,0.624858,0.623544,0.626172,0.581303,0.669892,10.0
9,sociopolitical,Total,0.221122,0.353061,0.988358,0.443620,0.443003,0.444238,0.418579,0.470586,10.0


social_physical


...

,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
60,inclu_suff_est,Mono,0.353999,0.435589,0.992923,2.078617,2.075582,2.081653,1.959505,2.204832,10.0
61,inclu_suff_est,One,0.427050,0.226017,0.981143,0.771869,0.769820,0.773918,0.716890,0.832208,10.0
62,inclu_suff_est,Two,0.263020,0.252680,0.984578,0.559341,0.558136,0.560547,0.520534,0.600601,10.0
63,inclu_suff_est,Three,0.201027,0.218139,0.971406,0.657435,0.656090,0.658780,0.612257,0.704770,10.0
64,inclu_suff_est,Total,0.360730,0.474391,0.992007,0.480975,0.480272,0.481679,0.453420,0.510230,10.0


cred_elect_est


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
65,cred_elect_est,Mono,0.341832,0.405415,0.988958,2.289923,2.286694,2.293152,2.155805,2.430111,10.0
66,cred_elect_est,One,0.145423,0.277802,0.978187,0.764760,0.762737,0.766783,0.710026,0.825084,10.0
67,cred_elect_est,Two,0.138705,0.251583,0.982481,0.486137,0.484899,0.487375,0.451281,0.523088,10.0
68,cred_elect_est,Three,0.181364,0.287823,0.974060,0.634541,0.633250,0.635832,0.590031,0.680900,10.0
69,cred_elect_est,Total,0.332777,0.381282,0.991471,0.436510,0.435893,0.437127,0.411304,0.463688,10.0


local_dem_est


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
70,local_dem_est,Mono,0.284999,0.395928,0.986609,2.121741,2.118755,2.124727,2.000400,2.247521,10.0
71,local_dem_est,One,0.278919,0.336822,0.983007,0.768554,0.766496,0.770611,0.713995,0.828699,10.0
72,local_dem_est,Two,0.129221,0.337371,0.994396,0.538809,0.537565,0.540053,0.502380,0.577771,10.0
73,local_dem_est,Three,0.206395,0.245730,0.967455,0.660466,0.659093,0.661840,0.615157,0.707947,10.0
74,local_dem_est,Total,0.285027,0.403067,0.991307,0.471191,0.470526,0.471856,0.444796,0.499795,10.0


In [16]:
dt = time.perf_counter() - t0
m, s = divmod(dt, 60)
print(f"Tiempo total: {int(m)} min {s:.1f} s")

Tiempo total: 5 min 8.9 s
